# Notebook 5.1 - Curation-Keywords

This notebook implements the workflow defined in:

[Curating keywords](https://gitlab.gwdg.de/sshoc/marketplace-curation/-/issues/1#note_71056) GitLab issue

The notebook works as follows:

0. Imports external libraries and loads the MP dataset and the google sheet
2. Updates keywords on MP as follows:
    1. Looks for vocabulary terms with the value from the column *Keyword to map*
    2. Looks for the term in the column *Map to*
    3. Goes through all MP items to look for the *keywords-to-map* in the keyword-dynamic-property
    4. Replaces the  the *keywords-to-map* in the MP dataset.


## 0 Requirements to run this notebook

This section gives all the relevant information to "interact" with the MP data.

### 0.1 libraries
*There are a number of external libraries needed to run the notebook* 

*Furthermore, a dedicated SSH Open Marketplace library - sshmarketplacelib - with customised functions has been created and can be imported using the python import commands.* 

*Below the libraries import needed to run this notebook*

In [1]:
import numpy as np
import pandas as pd
import requests
#import the MarketPlace Library 
from sshmarketplacelib import MPData as mpd
from sshmarketplacelib import  eval as eva, helper as hel

### 0.2 Get the data



Get the MarketPlace dataset

In [2]:
mpdata = mpd()
df_tool_flat =mpdata.getMPItems ("toolsandservices", True)
df_publication_flat =mpdata.getMPItems ("publications", True)
df_trainingmaterials_flat =mpdata.getMPItems ("trainingmaterials", True)
df_workflows_flat =mpdata.getMPItems ("workflows", True)
df_datasets_flat =mpdata.getMPItems ("datasets", True)

getting data from local repository...
getting data from local repository...
getting data from local repository...
getting data from local repository...
getting data from local repository...


Get the list of keywords from the [gsheet](https://docs.google.com/spreadsheets/d/1-Oh9_SxIhfMAT6KNJrMf4LetCpy5s1fHZEyTL__TUVA/edit#gid=0)

In [3]:
sheet_id = '1-Oh9_SxIhfMAT6KNJrMf4LetCpy5s1fHZEyTL__TUVA'
sheet_name = 'Mappings'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={sheet_name}'
df_keywords=pd.read_csv(url)

In [4]:
df_grouped_keyword=df_keywords.groupby(['Keyword to map'])['Map to'].apply(list).reset_index(name='Maps')

In [5]:
df_grouped_keyword.iloc[40:45]

,Keyword to map,Maps
40,"History, Philosophy and Sociology of Sciences",[https://vocabs.acdh.oeaw.ac.at/oefosdisciplin...
41,Information Retrieval > Analysis-Content Analysis,[https://vocabs.dariah.eu/tadirah/informationR...
42,Interpretation,[https://vocabs.dariah.eu/tadirah/interpreting]
43,Linguistics,[https://vocabs.acdh.oeaw.ac.at/oefosdisciplin...
44,Linked open data > Enrichment-Annotation; Diss...,"[https://vocabs.dariah.eu/tadirah/publishing ,..."


The function *getMPConcepts()* is a custom function that uses the API entry: 

GET https://marketplace-api.sshopencloud.eu/api/concept-search?perpage=100&q=URI

to get all the *concepts* from the MarketPlace dataset. 

**Note that this function may require a long execution time**



In [6]:
df_concepts=mpdata.getMPConcepts()

In [7]:
utils=hel.Util()
resultfields=['persistentId', 'MPUrl', 'category', 'label', 'type.code', 'type.label', 'concept.code', 'concept.label', 'concept.uri', 'concept.vocabulary.scheme']
udf_alprop=utils.getAllPropertiesBySources()
udf_alprop=udf_alprop.loc[ : ,resultfields]

A few lines of the gsheet

In [8]:
udf_alprop.head()

,persistentId,MPUrl,category,label,type.code,type.label,concept.code,concept.label,concept.uri,concept.vocabulary.scheme
0,SIU1nO,tool-or-service/SIU1nO,tool-or-service,140kit,mode-of-use,Mode of use,webApplication,Web application,https://vocabs.sshopencloud.eu/vocabularies/in...,https://vocabs.sshopencloud.eu/vocabularies/in...
1,SIU1nO,tool-or-service/SIU1nO,tool-or-service,140kit,activity,Activity,capturing,Capturing,https://vocabs.dariah.eu/tadirah/capturing,https://vocabs.dariah.eu/tadirah/
2,SIU1nO,tool-or-service/SIU1nO,tool-or-service,140kit,activity,Activity,dataVisualization,Data Visualization,https://vocabs.dariah.eu/tadirah/dataVisualiza...,https://vocabs.dariah.eu/tadirah/
3,SIU1nO,tool-or-service/SIU1nO,tool-or-service,140kit,activity,Activity,analyzing,Analyzing,https://vocabs.dariah.eu/tadirah/analyzing,https://vocabs.dariah.eu/tadirah/
4,SIU1nO,tool-or-service/SIU1nO,tool-or-service,140kit,activity,Activity,analyzing,Analyzing,https://vocabs.dariah.eu/tadirah/analyzing,https://vocabs.dariah.eu/tadirah/


In [9]:
#udf_alprop[udf_alprop['label']=='Manuscript Desk']

### 1 Update keywords

The function *getMPKeywordProperies(mKey)* used below is a custum function that uses the API entry 

    GET https://marketplace-api.sshopencloud.eu/api/concept-search?types=keyword&q=VALUE

and returns the vocabulary terms for *mKey*.  

The returned dataset is filtered to individuate those values coming from the vocabulary *sshoc-keyword* (vocabulary[code]=sshoc-keyword).


In [10]:
pd.options.mode.chained_assignment = None
selectedItems=pd.DataFrame()
#df_vocterms=pd.DataFrame()
for rown, row in df_grouped_keyword.iterrows():
    
    uk=df_grouped_keyword.iloc[rown]['Keyword to map']
    myKeys=df_grouped_keyword.iloc[rown]['Maps']
    #df_vocterms=pd.DataFrame()
    jsonmapto=[]
    for myKey in myKeys:
        myKey=myKey.strip()
        df_vocterms=mpdata.getMPKeywordProperties(myKey.strip())
        print (f'n. vocabulary terms: {df_vocterms.shape[0]}')
        print(f'\n***** checking {uk}, {myKey}\n')
        if (df_vocterms.empty):
            print (f"#####  No vocabulary terms found for {myKey[0]}")
            continue;
        df_vocterms.loc[:]=df_vocterms.loc[(df_vocterms.vocabulary=={'code': 'sshoc-keyword'}), ]
    
        #The set of vocabulary terms is filterd to individuate those with the exact match (case-insensitive) 
        df_vocterms=df_vocterms.loc[(df_vocterms.code).str.replace(' %3E ', ' > ').str.lower()==myKey.lower()]

        #Search for the one concept that has in the uri column the value of the Map to. There should be only one such concept.
    
   
        df_mapto=df_concepts.loc[(df_concepts.uri==myKey),]
       
    
        #Filter the dataset using the Keyword to Map

        df_items=udf_alprop.loc[(udf_alprop['concept.label'].str.lower()==uk.lower()), ]#& (udf_alprop['type.code'].str.lower()=='keyword')]
        
        print (f' n. prop: {df_items.shape[0]}')
        if (df_items.empty):
            print (f"\n%%%%%%%%  No items found for {myKey}")
            continue;
        print (f'\n&&&&&  Found as {df_items.iloc[0]["type.code"]}')
        #update the MP
        jsonConcept={}
        jsonConceptVal={}
    
        jsonConceptVal["code"]=df_mapto.iloc[0].code
        jsonConceptVal["vocabulary"]=df_mapto.iloc[0].vocabulary
        jsonConceptVal["uri"]=df_mapto.iloc[0].uri
        #jsonConcept
        
        attrList={}
        attrList["type"]=df_mapto.iloc[0].types[0]
        attrList["concept"]=jsonConceptVal
        jsonmapto.append(attrList)
        filterList={}
        filterList["concept"]=uk.lower()
        #print (f'update parameters: {attrList} - {filterList} \n')
    
    #df_items.loc[ : ,('updateList')]=[attrList for _ in range(df_items.shape[0])]
    df_items.loc[ : ,('updateList')]=[jsonmapto for _ in range(df_items.shape[0])]
    df_items.loc[ : ,('filterList')]=[filterList for _ in range(df_items.shape[0])]
    
    #selectedItems=selectedItems.append(df_items)
    selectedItems=pd.concat([selectedItems, df_items.loc[df_items.astype(str).drop_duplicates(keep='first').index]])
    

attrList={}
filterList={}
selectedItems.head()
mpdata.updateItemsProperties(selectedItems)


https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/concept-search?types=keyword&q=https%3A%2F%2Fvocabs.dariah.eu%2Ftadirah%2Faggregating
2
n. vocabulary terms: 47

***** checking Aggregation, https://vocabs.dariah.eu/tadirah/aggregating

 n. prop: 0

%%%%%%%%  No items found for https://vocabs.dariah.eu/tadirah/aggregating
https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/concept-search?types=keyword&q=https%3A%2F%2Fvocabs.dariah.eu%2Ftadirah%2Fanalyzing
1
n. vocabulary terms: 19

***** checking Analysis, https://vocabs.dariah.eu/tadirah/analyzing

 n. prop: 0

%%%%%%%%  No items found for https://vocabs.dariah.eu/tadirah/analyzing
https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/concept-search?types=keyword&q=https%3A%2F%2Fvocabs.dariah.eu%2Ftadirah%2Fannotating
5
n. vocabulary terms: 368

***** checking Annotating, https://vocabs.dariah.eu/tadirah/annotating

 n. prop: 282

&&&&&  Found as activity
https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/

1
n. vocabulary terms: 19

***** checking Cultural heritage and museology, https://vocabs.acdh.oeaw.ac.at/oefosdisciplines/605008

 n. prop: 9

&&&&&  Found as keyword
https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/concept-search?types=keyword&q=https%3A%2F%2Fvocabs.acdh.oeaw.ac.at%2Foefosdisciplines%2F605004
1
n. vocabulary terms: 19

***** checking Cultural studies, https://vocabs.acdh.oeaw.ac.at/oefosdisciplines/605004

 n. prop: 4

&&&&&  Found as discipline
https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/concept-search?types=keyword&q=https%3A%2F%2Fvocabs.acdh.oeaw.ac.at%2Foefosdisciplines%2F504006
1
n. vocabulary terms: 19

***** checking Demography, https://vocabs.acdh.oeaw.ac.at/oefosdisciplines/504006

 n. prop: 7

&&&&&  Found as discipline
https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/concept-search?types=keyword&q=https%3A%2F%2Fvocabs.sshopencloud.eu%2Fvocabularies%2Feosc-resource-category%2Fsubcategory-access_physical_and_einfrastruct

1
n. vocabulary terms: 19

***** checking History, Philosophy and Sociology of Sciences, https://vocabs.acdh.oeaw.ac.at/oefosdisciplines/603123

 n. prop: 7

&&&&&  Found as keyword
https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/concept-search?types=keyword&q=https%3A%2F%2Fvocabs.dariah.eu%2Ftadirah%2FinformationRetrieval
1
n. vocabulary terms: 19

***** checking Information Retrieval > Analysis-Content Analysis, https://vocabs.dariah.eu/tadirah/informationRetrieval

 n. prop: 0

%%%%%%%%  No items found for https://vocabs.dariah.eu/tadirah/informationRetrieval
https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/concept-search?types=keyword&q=https%3A%2F%2Fvocabs.dariah.eu%2Ftadirah%2FcontentAnalysis
1
n. vocabulary terms: 19

***** checking Information Retrieval > Analysis-Content Analysis, https://vocabs.dariah.eu/tadirah/contentAnalysis

 n. prop: 0

%%%%%%%%  No items found for https://vocabs.dariah.eu/tadirah/contentAnalysis
https://sshoc-marketplace-api-stage

1
n. vocabulary terms: 19

***** checking Photography, https://vocabs.acdh.oeaw.ac.at/oefosdisciplines/604012

 n. prop: 3

&&&&&  Found as discipline
https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/concept-search?types=keyword&q=https%3A%2F%2Fvocabs.sshopencloud.eu%2Fvocabularies%2Fmedia-type%2Fapplicationslashvnd.ms-powerpoint
2
n. vocabulary terms: 60

***** checking PowerPoint, https://vocabs.sshopencloud.eu/vocabularies/media-type/applicationslashvnd.ms-powerpoint

 n. prop: 0

%%%%%%%%  No items found for https://vocabs.sshopencloud.eu/vocabularies/media-type/applicationslashvnd.ms-powerpoint
https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/concept-search?types=keyword&q=https%3A%2F%2Fvocabs.dariah.eu%2Ftadirah%2FpreservationMetadata
1
n. vocabulary terms: 19

***** checking Preservation - Metadata > Storage - Preservation, https://vocabs.dariah.eu/tadirah/preservationMetadata

 n. prop: 2

&&&&&  Found as keyword
https://sshoc-marketplace-api-stage.acdh-de

2
n. vocabulary terms: 65

***** checking Video, https://vocabs.sshopencloud.eu/vocabularies/media-type/video

 n. prop: 89

&&&&&  Found as object-format
https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/concept-search?types=keyword&q=https%3A%2F%2Fvocabs.dariah.eu%2Ftadirah%2FdataVisualization
1
n. vocabulary terms: 19

***** checking Visualization, https://vocabs.dariah.eu/tadirah/dataVisualization

 n. prop: 0

%%%%%%%%  No items found for https://vocabs.dariah.eu/tadirah/dataVisualization
https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/concept-search?types=keyword&q=https%3A%2F%2Fvocabs.dariah.eu%2Fsshoc-keyword%2Fweb+development
1
n. vocabulary terms: 2

***** checking Web Development, https://vocabs.dariah.eu/sshoc-keyword/web development

 n. prop: 111

&&&&&  Found as activity
https://sshoc-marketplace-api-stage.acdh-dev.oeaw.ac.at/api/concept-search?types=keyword&q=https%3A%2F%2Fvocabs.dariah.eu%2Ftadirah%2FwebDevelopment
1
n. vocabulary terms: 19

*****

...

 {'type': {'code': 'keyword'}, 'concept': {'code': 'POS', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/POS'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'POS', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/POS'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'Metadata', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/Metadata'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'Metadata', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/Metadata'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'POS', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/POS'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'POS', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/POS'}} appended
 {'type'

 {'type': {'code': 'keyword'}, 'concept': {'code': 'POS', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/POS'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'POS', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/POS'}} appended
 {'type': {'code': 'object-format'}, 'concept': {'code': 'text', 'vocabulary': {'code': 'iana-media-type'}, 'uri': 'https://vocabs.sshopencloud.eu/vocabularies/media-type/text'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'NLP', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/NLP'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'POS', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/POS'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'POS', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/POS'}} appended
 {

 {'type': {'code': 'keyword'}, 'concept': {'code': 'NLP', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/NLP'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'POS', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/POS'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'POS', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/POS'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'Metadata', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/Metadata'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'OCR', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/OCR'}} appended
 {'type': {'code': 'language'}, 'concept': {'code': 'nor', 'vocabulary': {'code': 'iso-639-3'}, 'uri': 'https://vocabs.acdh.oeaw.ac.at/iso6393/nor'}} appended
 {'type': {'code': 'k

 {'type': {'code': 'keyword'}, 'concept': {'code': 'POS', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/POS'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'POS', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/POS'}} appended
 {'type': {'code': 'language'}, 'concept': {'code': 'nor', 'vocabulary': {'code': 'iso-639-3'}, 'uri': 'https://vocabs.acdh.oeaw.ac.at/iso6393/nor'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'POS', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/POS'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'NLP', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/NLP'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'NLP', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/NLP'}} appended
 {'type': {'code': 'keyword'}, 

 {'type': {'code': 'keyword'}, 'concept': {'code': 'POS', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/POS'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'POS', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/POS'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'POS', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/POS'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'POS', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/POS'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'POS', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/POS'}} appended
 {'type': {'code': 'keyword'}, 'concept': {'code': 'Metadata', 'vocabulary': {'code': 'sshoc-keyword'}, 'uri': 'https://vocabs.dariah.eu/sshoc-keyword/Metadata'}} appended
 {'type': {'code':

In [ ]:
df_concepts.sort_values('label').iloc[7670:7690]

{
      "type": {
        "code": "activity"
      },
      "concept": {
        "code": "informationRetrieval",
        "vocabulary": {
          "code": "tadirah2"
        },
        "uri": "https://vocabs.dariah.eu/tadirah/informationRetrieval"
      }
    },
    {
      "type": {
        "code": "activity"
      },
      "concept": {
        "code": "contentAnalysis",
        "vocabulary": {
          "code": "tadirah2"
        },
        "uri": "https://vocabs.dariah.eu/tadirah/contentAnalysis"
      }
    },

In [ ]:

pd.options.mode.chained_assignment = None
selectedItems=pd.DataFrame()
#df_vocterms=pd.DataFrame()
for rown, row in df_keywords[201:208].iterrows():
    uk=df_keywords.iloc[rown]['Keyword to map']
    myKey=df_keywords.iloc[rown]['Map to']
    #df_vocterms=pd.DataFrame()
    df_vocterms=mpdata.getMPKeywordProperties(myKey)
    print(f'\n***** checking {myKey}\n')
    if (df_vocterms.empty):
        print (f"#####  No vocabulary terms found for {myKey}")
        continue;
    df_vocterms.loc[:]=df_vocterms.loc[(df_vocterms.vocabulary=={'code': 'sshoc-keyword'}), ]
    
    #The set of vocabulary terms is filterd to individuate those with the exact match (case-insensitive) 
    df_vocterms=df_vocterms.loc[(df_vocterms.code).str.replace(' %3E ', ' > ').str.lower()==myKey.lower()]

    #Search for the one concept that has in the uri column the value of the Map to. There should be only one such concept.
    
   
    df_mapto=df_concepts.loc[(df_concepts.uri==df_keywords.iloc[rown]['Map to']),]

    
    #Filter the dataset using the Keyword to Map

    df_items=udf_alprop.loc[(udf_alprop['concept.label'].str.lower()==uk.lower()), ]#& (udf_alprop['type.code'].str.lower()=='keyword')]
    
    if (df_items.empty):
        print (f"\n%%%%%%%%  No Items found for {myKey}")
        continue;
    print (f'\n&&&&&  Found as {df_items.iloc[0]["type.code"]}')
    #update the MP
    jsonConcept={}
    jsonConceptVal={}
    
    jsonConceptVal["code"]=df_mapto.iloc[0].code
    jsonConceptVal["vocabulary"]=df_mapto.iloc[0].vocabulary
    jsonConceptVal["uri"]=df_mapto.iloc[0].uri
    #jsonConcept
   
    attrList={}
    attrList["type"]=df_mapto.iloc[0].types[0]
    attrList["concept"]=jsonConceptVal
    filterList={}
    filterList["concept"]=myKey.lower()
    print (f'update parameters: {attrList}, {filterList} \n')
    
    #df_items.loc[ : ,('updateList')]=[attrList for _ in range(df_items.shape[0])]
    df_items.loc[ : ,('updateList')]=[attrList for _ in range(df_items.shape[0])]
    df_items.loc[ : ,('filterList')]=[filterList for _ in range(df_items.shape[0])]
    
    #selectedItems=selectedItems.append(df_items)
    selectedItems=pd.concat([selectedItems, df_items])
    

attrList={}
filterList={}
selectedItems.head()
mpdata.updateItemsProperties(selectedItems)


In [ ]:
df_vocterms.head()